In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import math

from keras.preprocessing.text import *
from keras.preprocessing.sequence import skipgrams, pad_sequences

Using TensorFlow backend.


In [2]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

df_train = pd.read_csv('input/train.csv')

x_train = df_train['comment_text']
y_train = df_train[categories].as_matrix()

display(df_train.head())
m = x_train.shape[0]
display(f'total m = {m}')

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


'total m = 159571'

In [3]:
max_len = 300
epochs = 30
one_weight = 0.7

In [4]:
embedding_index = {}

f = open('input/glove.6B.300d.txt')
for line in f:
    values = line.split()
    w = values[0]
    weights = np.asarray(values[1:], dtype='float32')
    embedding_index[w] = weights
f.close()

In [5]:
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(texts=x_train)

vocabulary_size = len(tokenizer.word_index)
sequences = tokenizer.texts_to_sequences(texts=x_train)

max_seq_len = 0
for sequence in sequences:
    max_seq_len = max(max_seq_len, len(sequence))

print('max length {}'.format(max_seq_len))

min_seq_len = max_seq_len
for sequence in sequences:
    min_seq_len = min(min_seq_len, len(sequence))
    
print('min length {}'.format(min_seq_len))

max length 1403
min length 1


In [6]:
np.mean([len(x) for x in sequences])


68.221569082101382

In [7]:
embedding_dim = 300

embedding_matrix = np.zeros((vocabulary_size + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten
from keras.metrics import categorical_accuracy
from keras.layers import Bidirectional, LSTM, GRU, Dense

import keras.backend as K
import tensorflow as tf

embedding = Embedding(vocabulary_size + 1, embedding_dim, weights=[embedding_matrix], trainable=False)

def init_weighted_binary_crossentropy(one_weights, zero_weights):

    def weight_binary_crossentropy(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), (1.0 - K.epsilon()))
        
        o_terms = tf.multiply(tf.multiply(y_true, tf.log(y_pred)), one_weights)
        z_terms = tf.multiply(tf.subtract(1.0, y_true), tf.log(tf.subtract(1.0, y_pred)))

        terms = tf.add(o_terms, z_terms)
        terms = tf.negative(K.mean(terms))

        return K.mean(terms)
    
    return weight_binary_crossentropy

input_model = Sequential()
input_model.add(embedding)
input_model.add(Bidirectional(LSTM(32)))
input_model.add(Dense(6, activation='sigmoid'))

c_count = [15294, 1595, 8449, 478, 7877, 1405]
o_weights = [max(1.0, math.log(one_weight * m / float(c))) for c in c_count]
print(o_weights)
z_weights = np.zeros(6)

input_model.compile(optimizer='adam', loss=init_weighted_binary_crossentropy(o_weights, z_weights), metrics=[categorical_accuracy])

[1.9883534256840718, 4.248940284069297, 2.5817659291353974, 5.4539585667967865, 2.6518668995849177, 4.375776717520396]


In [9]:
batch_size = 3072

def inputs():
    padded_sequences = pad_sequences(sequences=sequences, maxlen=max_len, padding='post')
    x_list = []
    y_list = []
    for sequence, y_train_val in zip(padded_sequences, y_train):
        x_list.append(sequence)
        y_list.append(y_train_val)
    return np.array(x_list), np.array(y_list)

x_inputs, y_inputs = inputs()

hist = input_model.fit(x=x_inputs, y=y_inputs, epochs=epochs, batch_size=batch_size, shuffle=True)

print(hist.history.items())


Epoch 1/30
159571/159571 [==============================] - 845s 5ms/step - loss: 0.4250 - categorical_accuracy: 0.1718
Epoch 2/30
159571/159571 [==============================] - 806s 5ms/step - loss: 0.2315 - categorical_accuracy: 0.8251
Epoch 3/30
159571/159571 [==============================] - 801s 5ms/step - loss: 0.1588 - categorical_accuracy: 0.7245
Epoch 4/30
159571/159571 [==============================] - 829s 5ms/step - loss: 0.1308 - categorical_accuracy: 0.9300
Epoch 5/30
159571/159571 [==============================] - 818s 5ms/step - loss: 0.1177 - categorical_accuracy: 0.9704
Epoch 6/30
159571/159571 [==============================] - 806s 5ms/step - loss: 0.1101 - categorical_accuracy: 0.9754
Epoch 7/30
159571/159571 [==============================] - 806s 5ms/step - loss: 0.1049 - categorical_accuracy: 0.9680
Epoch 8/30
159571/159571 [==============================] - 807s 5ms/step - loss: 0.1012 - categorical_accuracy: 0.9677
Epoch 9/30
159571/159571 [==============

In [28]:
hist = input_model.fit(x=x_inputs, y=y_inputs, epochs=5, batch_size=batch_size, shuffle=True)


y_pred = input_model.predict(x=x_inputs[0:1000, :])

Epoch 1/5
159571/159571 [==============================] - 1795s 11ms/step - loss: 0.0671 - categorical_accuracy: 0.9766
Epoch 2/5
159571/159571 [==============================] - 1644s 10ms/step - loss: 0.0665 - categorical_accuracy: 0.9752
Epoch 3/5
159571/159571 [==============================] - 1653s 10ms/step - loss: 0.0663 - categorical_accuracy: 0.9772
Epoch 4/5
159571/159571 [==============================] - 24452s 153ms/step - loss: 0.0656 - categorical_accuracy: 0.9729
Epoch 5/5
159571/159571 [==============================] - 913s 6ms/step - loss: 0.0652 - categorical_accuracy: 0.9733


In [29]:
cat_max = [np.max(y_pred[:,i]) for i in range(len(categories))]

display(cat_max)

display(y_pred)

y_pred_labels = np.zeros((y_pred.shape[0], y_pred.shape[1]))

for cmax, i in zip(cat_max, range(len(categories))):
    y_pred_labels[y_pred[:,i] >= cmax, i] = 1
    y_pred_labels[y_pred[:,i] < cmax, i] = 0


[0.99958831, 0.92318612, 0.99542254, 0.92499238, 0.98819602, 0.95567679]

array([[  1.34259509e-03,   1.01660269e-04,   5.04357798e-04,
          5.65093578e-05,   2.85132468e-04,   9.01358799e-05],
       [  1.12030422e-03,   1.06385640e-04,   4.41085751e-04,
          8.02847499e-05,   3.29066068e-04,   1.43011464e-04],
       [  8.89734626e-02,   7.18377822e-04,   2.27558482e-02,
          1.84875773e-03,   2.28070728e-02,   1.61362044e-03],
       ..., 
       [  6.98640360e-04,   8.04669326e-05,   5.92404976e-04,
          3.76475000e-05,   3.88850458e-04,   9.50718822e-05],
       [  6.00220524e-02,   3.56231554e-04,   8.33033677e-03,
          6.67122426e-04,   4.94798832e-03,   7.93009764e-04],
       [  4.79182228e-02,   4.13200294e-04,   4.67344839e-03,
          8.76317674e-04,   5.98357385e-03,   1.57622644e-03]], dtype=float32)

In [30]:
display(y_pred_labels)



array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train[0:1000,:], y_pred_labels)

0.89200000000000002

In [32]:
df_test = pd.read_csv('input/test.csv')

x_test = df_test['comment_text']

print(x_test.shape)

(153164,)


In [33]:
sequences_test = tokenizer.texts_to_sequences(x_test)
padded_sequences_test = pad_sequences(sequences=sequences_test, maxlen=max_len, padding='post')

print(len(padded_sequences_test))

153164


In [34]:
y_test_pred = input_model.predict(padded_sequences_test)

In [35]:
print(y_test_pred.shape)

test_cat_max = [np.max(y_test_pred[:,i]) for i in range(len(categories))]

print(test_cat_max)

test_cat_min = [np.min(y_test_pred[:,i]) for i in range(len(categories))]

print(test_cat_min)

(153164, 6)
[0.99993396, 0.99841452, 0.99944335, 0.99987817, 0.99900961, 0.99006492]
[7.2540293e-05, 5.5869591e-06, 7.8540055e-05, 2.1909384e-06, 1.3558042e-05, 3.8137257e-06]


In [36]:
y_test_pred_labels = np.zeros((y_test_pred.shape[0], y_test_pred.shape[1]))

print(y_test_pred_labels.shape)

(153164, 6)


In [37]:
threshold = [0.90, 0.90, 0.90, 0.90, 0.90, 0.90]

for t, i in zip(threshold, range(len(threshold))):
    y_test_pred_labels[y_test_pred[:,i] >= t, i] = 1

display(y_test_pred_labels)

array([[ 1.,  0.,  1.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.,  0.]])

In [38]:
df_submission = pd.DataFrame()
df_submission['id'] = df_test['id']
for c, i in zip(categories, range(len(categories))):
    df_submission[c] = y_test_pred_labels[:, i]

    
print(df_submission.shape)

df_submission.to_csv('output/lstm_glve_4B_07_300_35e_submission.csv',index=False)

(153164, 7)
